## Flight refund process

In [1]:

from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [2]:
model_client = azure_openai_chat_completion_client

travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)


In [3]:
task = "항공권을 환불해야 합니다."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


await run_team_stream()

---------- TextMessage (user) ----------
항공권을 환불해야 합니다.
---------- ThoughtEvent (travel_agent) ----------
항공권 환불을 도와드리겠습니다. 환불을 원하시는 항공편 정보(예: 항공사, 예약 번호, 출발일 등)를 알려주시겠어요?
---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_EPKjhBdOD0sgyxpqhiY0sRhG', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', name='transfer_to_user', call_id='call_EPKjhBdOD0sgyxpqhiY0sRhG', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to user, adopting the role of user immediately.
---------- HandoffMessage (user) ----------
A10
---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_PH3gO6U4tgROrB9ST2jNWy8f', arguments='{}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_

## Stock Research

In [5]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "테슬라, 사이버트럭 생산 확대",
            "date": "2024-03-20",
            "summary": "Tesla는 강력한 수요를 충족하기 위해 기가팩토리 텍사스의 사이버트럭 제조 역량을 강화합니다.",
        },
        {
            "title": "가능성을 보여주는 테슬라 FSD 베타 버전",
            "date": "2024-03-19",
            "summary": "최신 완전 자율주행 베타 버전에서는 도심 내비게이션 및 안전 기능이 크게 개선되었습니다.",
        },
        {
            "title": "Model Y, 글로벌 전기차 판매 1위",
            "date": "2024-03-18",
            "summary": "Tesla의 Model Y가 전 세계에서 가장 많이 팔린 전기자동차가 되어 상당한 시장 점유율을 차지했습니다.",
        },
    ]

In [6]:
model_client = azure_openai_chat_completion_client
planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [7]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "TSLA 주식에 대한 시장 조사 수행"
response = await Console(research_team.run_stream(task=task))

---------- TextMessage (user) ----------
TSLA 주식에 대한 시장 조사 수행


---------- ThoughtEvent (planner) ----------
시장 조사를 체계적으로 수행하기 위한 계획을 제시하겠습니다.

1. 금융 분석: TSLA 주식의 최신 주가 동향 및 재무 지표 분석
2. 뉴스 수집 및 분석: TSLA 관련 최신 뉴스와 시장 반응 파악
3. 보고서 작성: 분석 결과를 종합하여 최종 시장 조사 보고서 작성

먼저, TSLA 주식의 금융 데이터를 분석하도록 금융 분석 전문가에게 진행을 요청하겠습니다.
---------- ToolCallRequestEvent (planner) ----------
[FunctionCall(id='call_NlCqLWrpx7x0HDIi4stJj18B', arguments='{}', name='transfer_to_financial_analyst')]
---------- ToolCallExecutionEvent (planner) ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', name='transfer_to_financial_analyst', call_id='call_NlCqLWrpx7x0HDIi4stJj18B', is_error=False)]
---------- HandoffMessage (planner) ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- ToolCallRequestEvent (financial_analyst) ----------
[FunctionCall(id='call_ZRPIlt3cPTJLdAIWomZqHQZP', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
---------- ToolCal

In [8]:
print(response.messages[-7].content)

[FunctionExecutionResult(content='Transferred to writer, adopting the role of writer immediately.', name='transfer_to_writer', call_id='call_r3sk0yrhEGKtBoMHnkIH4NSw', is_error=False)]
